## extracting overture data using the API

In [12]:
import duckdb

duckdb.install_extension('httpfs')
duckdb.load_extension('httpfs')

duckdb.sql("""
SET s3_region='us-west-2';
SET s3_access_key_id='';
SET s3_secret_access_key='';
SET s3_session_token='';
SET s3_url_style='path';
SET s3_use_ssl=true;
""")

# Try to read file names from the public Overture S3 bucket
df_files = duckdb.sql("""
SELECT DISTINCT filename
FROM read_parquet(
    's3://overturemaps-us-west-2/release/2025-10-22.0/theme=base/type=infrastructure/*.parquet',
    filename=true
)
LIMIT 10;
""").df()

print(df_files)


                                            filename
0  s3://overturemaps-us-west-2/release/2025-10-22...
1  s3://overturemaps-us-west-2/release/2025-10-22...
2  s3://overturemaps-us-west-2/release/2025-10-22...
3  s3://overturemaps-us-west-2/release/2025-10-22...
4  s3://overturemaps-us-west-2/release/2025-10-22...
5  s3://overturemaps-us-west-2/release/2025-10-22...
6  s3://overturemaps-us-west-2/release/2025-10-22...
7  s3://overturemaps-us-west-2/release/2025-10-22...
8  s3://overturemaps-us-west-2/release/2025-10-22...
9  s3://overturemaps-us-west-2/release/2025-10-22...
